# [Common Random Variables](@id CommonRandomVariables)
Polynomial chaos expansion (PCE) is a Hilbert space technique for random variables with finite variance.
Mathematically equivalent to Fourier series expansions for periodic signals, PCE allows to characterize a random variable in terms of its PCE coefficients (aka Fourier coefficients).
That is, the PCE of a random variable $\mathsf{x}$ is given by
$$
\mathsf{x} = \sum_{i=0}^L x_i \phi_i,
$$
where $x_i$ are the so-called PCE coefficients, and $\phi_i$ are the orthogonal polynomials that are orthogonal relative to the probability density function of $\mathsf{x}$.

This tutorial walks you through the PCE of common random variables, namely Gaussian (`gaussian`), Beta (`beta01`), Uniform(`uniform01`), Logistic (`logistic`), and shows how they are implemented in `PolyChaos`.

## Construction of Basis

We begin by specifying the names and, if any, parameters for the uncertainties.

In [1]:
using PolyChaos
α, β = 1.3, 2.2
polynames = ["gaussian", "beta01", "uniform01", "logistic"]
pars = [Dict(), Dict(:shape_a=>α,:shape_b=>β), Dict(), Dict()]

4-element Array{Dict,1}:
 Dict{Any,Any}()                  
 Dict(:shape_b=>2.2,:shape_a=>1.3)
 Dict{Any,Any}()                  
 Dict{Any,Any}()                  

The orthogonal polynomials are constructed using `OrthoPoly` (here of degree at most `d`, and stored in the dictionary `myops`).

In [2]:
d = 6
myops = Dict()
for (i,name) in enumerate(polynames)
    myops[name]=OrthoPoly(name,d,pars[i])
end

For example, let's evaluate the Gaussian basis polynomials at some points

In [3]:
points, degrees = randn(10), 0:2:d
op_gauss=myops["gaussian"]
[ evaluate(degree,points,op_gauss) for degree in degrees ]

4-element Array{Array{Float64,1},1}:
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]                                                     
 [1.55194, -0.884183, -0.999494, -0.991697, -0.850439, -0.682847, -0.999242, 3.76605, -0.706027, 3.9381]
 [-5.79924, 2.31851, 2.99696, 2.95025, 2.125, 1.19767, 2.99546, -2.88107, 1.32258, -2.24375]            
 [18.7706, -9.98788, -14.9772, -14.6274, -8.60193, -2.20499, -14.9659, -32.9943, -3.0421, -38.1434]     

If a quadrature rule is required, this can be added by calling `OrthoPolyQ`

In [4]:
myopqs = Dict()
for (i,name) in enumerate(polynames)
    myopqs[name]=OrthoPolyQ(name,d,pars[i])
end

## Finding PCE Coefficients
Having constructed the orthogonal bases, the question remains how to find the PCE coefficients for the common random variables.
Every random variable can be characterized exactly by two PCE coefficients.
For a Gaussian random variable, this is familiar: the mean and the variance suffice to describe a Gaussian random variable entirely.
The same is true for any random variable of finite variance given the right basis.
The function `convert2affinePCE` provides the first two PCE coefficients (hence the name affine) for the common random variables.

### Gaussian
Given the Gaussian random variable $\mathsf{x} \sim \mathcal{N}(\mu, \sigma^2)$ with $\sigma > 0$, the affine PCE coefficients are

In [5]:
# Gaussian
μ, σ = 2., 0.2
pce_gaussian = convert2affinePCE("gaussian",μ,σ)
# Uniform

2-element Array{Float64,1}:
 2.0
 0.2

### Uniform
Given the uniform random variable $\mathsf{x} \sim \mathcal{U}(a, b)$ with finite support $a<b$, the affine PCE coefficients are

In [6]:
a, b = -0.3, 1.2
convert2affinePCE("uniform01",a,b)

2-element Array{Float64,1}:
 0.45
 1.5 

Instead, if the expected value and standard deviation are known, the affine PCE coefficients of the uniform random variable are

In [7]:
pce_uniform = convert2affinePCE("uniform01",μ,σ;kind=:μσ)
# notice that the zero-order coefficient IS equivalent to the expected value μ

2-element Array{Float64,1}:
 2.0               
 0.6928203230275509

### Beta
Given the Beta random variable $\mathsf{x} \sim \mathcal{B}(a, b, \alpha, \beta)$ with finite support $a<b$ and shape parameters $\alpha, \beta > 0$, the affine PCE coefficients are

In [8]:
convert2affinePCE("beta01",a,b,Dict(:shape_a=>α,:shape_b=>β))

2-element Array{Float64,1}:
 0.2571428571428572
 1.5               

Instead, if the expected value and standard deviation are known, the affine PCE coefficients of the uniform random variable are

In [9]:
pce_beta = convert2affinePCE("beta01",μ,σ,Dict(:shape_a=>α,:shape_b=>β); kind=:μσ)

2-element Array{Float64,1}:
 2.0               
 0.8780541105074454

### Logistic

Given the logstic random variable $\mathsf{x} \sim \mathcal{L}(a_1,a_2)$ where $a_2>0$ with the probability density function
$$
\rho(t) = \frac{1}{4 a_2} \, \operatorname{sech}^2 \left(\frac{t-a_1}{2a_2}\right)
$$
the affine PCE coefficients of the uniform random variable are

In [10]:
a1, a2 = μ, sqrt(3)*σ/pi
pce_logistic = convert2affinePCE("logistic",a1,a2)

2-element Array{Float64,1}:
 2.0                
 0.11026577908435842

## Moments
It is a key feature of PCE to compute moments from the PCE coefficients alone; no sampling is required.

### Gaussian

In [11]:
mean(pce_gaussian,myops["gaussian"]), std(pce_gaussian,myops["gaussian"])

(2.0, 0.2)

### Uniform

In [12]:
mean(pce_uniform,myops["uniform01"]), std(pce_uniform,myops["uniform01"])

(2.0, 0.19999999999999998)

### Beta

In [13]:
mean(pce_beta,myops["beta01"]), std(pce_beta,myops["beta01"])

(2.0, 0.20000000000000004)

### Logistic

In [14]:
mean(pce_logistic,myops["logistic"]), std(pce_logistic,myops["logistic"])

(2.0, 0.2)

## Sampling
Having found the PCE coefficients, it may be useful to sample the random variables.
That means, find $N$ realizations of the random variable that obey the random variable's probability density function.
This is done in two steps:
1. Draw $N$ samples from the measure (`sampleMeasure()`), and then
2. Evaluate the basis polynomials and multiply times the PCE coefficients, i.e. $\sum_{i=0}^L x_i \phi_i(\xi_j)$ where $\xi_j$ is the $j$-th sample from the measure (`evaluatePCE()`).

Both steps are combined in the function `samplepCE()`.

### Gaussian

In [15]:
using Statistics
N = 1000
ξ_gaussian = sampleMeasure(N,myops["gaussian"])
samples_gaussian = evaluatePCE(pce_gaussian,ξ_gaussian,myops["gaussian"])
# samplePCE(N,pce_gaussian,myops["gaussian"])

1000-element Array{Float64,1}:
 2.1907661087520136
 1.663042602332315 
 1.9793196182874786
 2.054571660450479 
 1.6892770287109173
 1.8785011785994126
 1.939678975096966 
 1.5058767650163523
 2.1652237931936766
 2.1710816714077197
 2.0305330812359506
 2.3324470819265284
 1.8972919389968446
 ⋮                 
 1.9083936525475467
 2.149873461138305 
 2.288037374438226 
 2.018016049722023 
 1.8313991949469228
 1.8040936222927373
 1.9650218372924761
 2.0129875294704225
 1.6649137246077148
 2.2797879989684215
 2.0753834089248495
 2.162282978619439 

### Uniform

In [16]:
ξ_uniform = sampleMeasure(N,myops["uniform01"])
samples_uniform = evaluatePCE(pce_uniform,ξ_uniform,myops["uniform01"])
# samples_uniform = samplePCE(N,pce_uniform,myops["uniform01"])

1000-element Array{Float64,1}:
 2.182246590981403 
 1.8660407626221835
 1.7348542980700548
 2.0510770573299877
 2.0656209910473584
 2.2826390664891334
 2.3247562789591094
 1.9596403642982063
 2.010736711462756 
 2.2303499489736387
 1.8748482798451551
 1.7510445075338057
 1.9092835746639136
 ⋮                 
 1.8822711487131847
 1.8244481788622244
 1.7939152903021358
 1.6955585078163846
 1.808113559823288 
 1.699243254065136 
 1.9851223994266485
 1.9962403483510394
 2.21689028326404  
 2.197131021388452 
 1.9238920675523183
 1.9912126756774329

### Beta

In [17]:
ξ_beta = sampleMeasure(N,myops["beta01"])
samples_beta = evaluatePCE(pce_beta,ξ_beta,myops["beta01"])
# samples_beta = samplePCE(N,pce_beta,myops["beta01"])

1000-element Array{Float64,1}:
 1.718590753300027 
 2.008869867204546 
 1.9295231182184724
 2.084510946990528 
 2.147358149410141 
 2.1759480431080402
 1.8327019486628064
 2.116730279564404 
 1.743229471186151 
 2.2859934097975745
 1.7450979537588263
 2.083354580401723 
 1.8577442000334639
 ⋮                 
 1.732754314019092 
 2.0343866216525734
 2.298374947552816 
 2.2856254098595974
 2.4602794511559387
 1.990844015695832 
 1.8117715322357268
 1.9629049375921117
 2.10512961673262  
 1.8862805984752367
 1.7083611271432226
 2.1593596194783986

### Logistic

In [18]:
ξ_logistic = sampleMeasure(N,myops["logistic"])
samples_logistic = evaluatePCE(pce_logistic,ξ_logistic,myops["logistic"])
# samples_logistic = samplePCE(N,pce_logistic,myops["logistic"])

1000-element Array{Float64,1}:
 1.806487842107241 
 2.0469637991532372
 1.9295487062293468
 2.390120894497393 
 2.2171062440832525
 2.1823147376488574
 1.8325996564765532
 1.8664992036390193
 2.3704972043071533
 1.920199757116314 
 2.0032372242924237
 2.12506113303165  
 1.8835238042121722
 ⋮                 
 1.8829645443326795
 1.9828068519196267
 1.8157462224656036
 2.006314777868817 
 2.057138553994138 
 1.7115811150916103
 2.130525171203205 
 2.1605139125847175
 1.9563654667934631
 1.7775778694429076
 1.745048729513444 
 1.9568897676693622